# Pixelate map
This notebook mainly draw a pixelated map by formatting a json file  
The NASA data used in this example can be downloaded here:  
https://drive.google.com/file/d/17dWJQ_UxAJEYCpnw6n7bZ8aqa05DvCZT/view?usp=sharing

In [1]:
import sys
sys.path.append('..')

import json
import os

import numpy as np
import pandas as pd

from ev_temp_map.utils import get_lat_lon_mask
from ev_temp_map.utils import get_temperature
from ev_temp_map.utils import get_zone

## Get some data for the map
Before we draw the map, we need to have some data. Here I just copy-paste codes to count max consecutive MPID, calculate an EV score, and calculate maximum daily range loss  
Adding more data is trivia if you follow the same paradigm of formatting the data  

Define some constants

In [2]:
DATA_FILE_DIR = "./data/nasa/"

START_YEAR, END_YEAR = 2010, 2020

NUM_OF_YEARS = END_YEAR - START_YEAR

NUM_OF_MONTHS = 12

NUM_OF_DAYS = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31,
               9: 30, 10: 31, 11: 30, 12: 31}

First, We need to use any of the data files to extract latitudes, longitudes and the land-ocean mask

In [3]:
LAT, LON, MASK = get_lat_lon_mask(DATA_FILE_DIR+"20200101.nc4")

Data 1: maximum consecutive MPID (must plug-in day)  
Algorithm: Keep two counters. One records the current consec MPID, and the other records the max consec MPID it has seen so far. After counting one day's MPID, compare the two and keep the larger one. Use `np.where` to adapt this algo to count on arrays  
  
Data 2: average maximum and daily range loss

In [4]:
%%time
# read the scale factors
SCALE_FACTORS = pd.read_csv("./data/extrapolated_factors.csv")
PERCENT_LOSS = SCALE_FACTORS["Range Loss"].to_numpy()

# max_MPID will record the max MPID of all places in a year
max_MPID = np.ndarray(shape=(len(LAT), len(LON)))
# curr_MPID will count the consecutive MPID we have seen so far (the counter)
curr_MPID = np.ndarray(shape=(len(LAT), len(LON)))
# each_year_avg_loss will record each year's avgerage daily range loss
each_year_avg_loss = np.zeros(shape=(NUM_OF_YEARS, len(LAT), len(LON)))
# max_loss will record the maximum daily range loss
max_loss = np.zeros(shape=(len(LAT), len(LON)))

for year in range(START_YEAR, END_YEAR):
    print(year, end=' ')

    # keep track of the number of days
    i = 0
    # yearly_loss will record each day's range loss of this year
    yearly_loss = np.zeros(shape=(365, len(LAT), len(LON)))

    for month in range(1, NUM_OF_MONTHS+1):
        for day in range(1, NUM_OF_DAYS[month]+1):
            date = "{}{:02d}{:02d}".format(year, month, day)
            filepath = DATA_FILE_DIR + date + '.nc4'
            date_temp = get_temperature(filepath)

            # if this place has MPID on this day (temp<253.15K),
            # then curr_MPID+1
            # else, this place has no MPID on this day, which means not
            # consecutive, so we reset the counter to 0;
            curr_MPID = np.where(date_temp < 253.15, curr_MPID+1, 0)
            # this is equivalent to A = max(A, B)
            max_MPID = np.where(curr_MPID > max_MPID, curr_MPID, max_MPID)

            # get the range loss of each day in this year
            date_temp = np.round(date_temp-273.15, decimals=1)  # convert to oC
            # use the temperature difference as index. e.g. if temperature is
            # -12.5 oC, then its range loss will be the (-12.5+100)*10=875th
            # element in the percent_loss array
            # "+100" means "-(-100)", "*10" means "/0.1"
            index = (date_temp+100)*10
            yearly_loss[i] = PERCENT_LOSS[index.astype(int)]

            i += 1

    # calculate the yearly average daily range loss
    each_year_avg_loss[year-START_YEAR] = yearly_loss.mean(axis=0)
    # calculate the yearly maximum daily range loss
    # NOTE: range loss is a negative value, so we use min()
    yearly_max_loss = yearly_loss.min(axis=0)
    max_loss = np.where(yearly_max_loss < max_loss, yearly_max_loss, max_loss)

print('Finished!\n')

2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 Finished!

Wall time: 5min 34s


Data 3: EV score  
EV score is simply converted from daily range loss

In [5]:
# get the daily average range loss
avg_loss = each_year_avg_loss.mean(axis=0)

# round the maximum daily range loss to 1 decimal
max_loss = np.round(max_loss, decimals=1)

# calculate score and round it to 1 decimal
avg_score = (avg_loss-avg_loss.min())/(avg_loss.max()-avg_loss.min())*100
avg_score = np.round(avg_score, decimals=1)

## Build the map with pixels

Format a JSON file  
Each pixel is a rectangle shape (Polygon)

In [6]:
%%time
json_file = {'features': [], 'type': 'FeatureCollection'}
for i in range(len(LAT)):
    for j in range(len(LON)):
        if (not MASK[i, j]):
            feature = {'geometry': {'coordinates': [], 'type': 'Polygon'},
                       'properties': {},
                       'type': 'Feature'}

            # the shape boundary of one pixel. It is represented in a list
            pixel = [[[LON[j]-.125, LAT[i]-.125],
                      [LON[j]-.125, LAT[i]+.125],
                      [LON[j]+.125, LAT[i]+.125],
                      [LON[j]+.125, LAT[i]-.125]]]
            feature['geometry']['coordinates'] = pixel

            # if you want to add more info/values just follow this tempLATe:
            # feature['properties']['YOUR FEATURE NAME'] = YOUR VALUE
            feature['properties']['EV Zone'] = get_zone(avg_score[i, j])
            feature['properties']['Score'] = avg_score[i, j]
            feature['properties']['Max consecutive MPID'] = int(max_MPID[i, j])
            feature['properties']['Max daily range loss'] = max_loss[i, j]

            # append this pixel to the feature list
            json_file['features'].append(feature)

Wall time: 9.95 s


save the map in json format

In [7]:
os.makedirs("./geojson_files", exist_ok=True)
with open('./geojson_files/pixel_data.json', 'w') as outfile:
    json.dump(json_file, outfile)